# 0. Setup
Run this cell to set up some helpful tools.

In [13]:
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

log_data = []
P1 = True
P2 = True

def nCr(n): return n * (n - 1) # / 2

def log(p1_cutoff, p2_cutoff, p1_expected_payout, p2_expected_payout, time_taken, verbose=True):
    log_data.append({
        'p1_cutoff': p1_cutoff,
        'p2_cutoff': p2_cutoff,
        'p1_expected_payout': p1_expected_payout,
        'p2_expected_payout': p2_expected_payout,
        'time_taken': time_taken
    })

    message = '''
P1 Cutoff: {0}
P2 Cutoff: {1}
P1 Payout: {2:.5f}
P2 Payout: {3:.5f}
Executed in {4:.5f} seconds
'''.format(p1_cutoff,
            p2_cutoff,
            p1_expected_payout,
            p2_expected_payout,
            time_taken)

    if verbose: boxed(message)

def boxed(text):
    lines = text.splitlines()
    if lines and lines[0] == "": lines = lines[1:]
    if not lines: return

    max_width = max(map(len, lines))

    print("┌" + "─" * (max_width + 2) + "┐")
    for line in lines:
      print(f"│ {line.ljust(max_width)} │")
    print("└" + "─" * (max_width + 2) + "┘")

def graph_3D(p1=False, p2=False):
    df = pd.DataFrame(log_data)
    fig = go.Figure()

    if p1:
        fig.add_trace(go.Scatter3d(
            x=df['p1_cutoff'],
            y=df['p2_cutoff'],
            z=df['p1_expected_payout'],
            mode='markers',
            marker=dict(color='blue', size=4),
            name='Player 1'
        ))

    if p2:
        fig.add_trace(go.Scatter3d(
            x=df['p1_cutoff'],
            y=df['p2_cutoff'],
            z=df['p2_expected_payout'],
            mode='markers',
            marker=dict(color='red', size=4),
            name='Player 2'
        ))

    fig.update_layout(
        title='Payouts as a function of Cutoffs | Scatter',
        scene=dict(
            xaxis_title='P1 Cutoff',
            yaxis_title='P2 Cutoff',
            zaxis_title='Payout'
        ),
        showlegend=True
    )

    fig.show()

def graph_contour(p1=False, p2=False):
    df = pd.DataFrame(log_data).groupby(['p1_cutoff', 'p2_cutoff'], as_index=False).mean()
    fig = go.Figure()

    x = np.unique(df['p1_cutoff'])
    y = np.unique(df['p2_cutoff'])
    X, Y = np.meshgrid(x, y)

    if p1:
        Z1 = df.pivot_table(index='p2_cutoff', columns='p1_cutoff', values='p1_expected_payout').values
        fig.add_trace(go.Surface(x=X, y=Y, z=Z1, colorscale='Blues', opacity=0.9, name="Player 1 Payout"))

    if p2:
        Z2 = df.pivot_table(index='p2_cutoff', columns='p1_cutoff', values='p2_expected_payout').values
        fig.add_trace(go.Surface(x=X, y=Y, z=Z2, colorscale='Reds', opacity=0.9, name="Player 2 Payout"))

    fig.update_layout(
        title='Payouts as a function of Cutoffs | Contour',
        scene=dict(
            xaxis_title='P1 Cutoff',
            yaxis_title='P2 Cutoff',
            zaxis_title='Payout'
        ),
        showlegend=True
    )

    fig.show()

# 1. Simulation
Simulating payoffs for various cutoff combinations.

In [21]:
#@title { vertical-output: true }

n = 100
test_range = 100
p1_expected_payout = 0
p2_expected_payout = 0
trials = nCr(n)

def test_cutoffs(p1_cutoff, p2_cutoff):

  start_time = time.time()

  global p1_expected_payout
  global p2_expected_payout

  p1_expected_payout = 0
  p2_expected_payout = 0

  for p1_card in range(1, n + 1):
    for p2_card in range(1, n + 1):

      if p1_card == p2_card: continue

      if p1_card >= p1_cutoff and p2_card >= p2_cutoff:
        if p1_card > p2_card:
          p1_expected_payout += 2
          p2_expected_payout -= 2
        else:
          p1_expected_payout -= 2
          p2_expected_payout += 2

      elif p1_card < p1_cutoff and p2_card < p2_cutoff:
        if p1_card > p2_card:
          p1_expected_payout += 1
          p2_expected_payout -= 1
        else:
          p1_expected_payout -= 1
          p2_expected_payout += 1

      elif p1_card >= p1_cutoff:
        p1_expected_payout += 1
        p2_expected_payout -= 1

      elif p2_card >= p2_cutoff:
        p1_expected_payout -= 1
        p2_expected_payout += 1

  return time.time() - start_time

log_data = []
start_time = time.time()

for p1_test_cutoff in range(1, test_range + 1):
  for p2_test_cutoff in range(1, test_range + 1):

    log(p1_test_cutoff,
        p2_test_cutoff,
        p1_expected_payout / trials,
        p2_expected_payout / trials,
        test_cutoffs(p1_test_cutoff, p2_test_cutoff),
        verbose=False)

print(f"Executed all iterations in {round(time.time() - start_time, 3)} seconds.\n")
graph_3D(P1, P2)
graph_contour(P1, P2)

Executed all iterations in 14.648 seconds.

